In [15]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from imblearn.under_sampling import RandomUnderSampler
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import tqdm
from sklearn import preprocessing
from keras.preprocessing import text, sequence
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn import decomposition
from gensim.models import Word2Vec
import gensim

In [3]:
from corpus import load_corpus, save_corpus
corpus = load_corpus()
corpus = corpus.astype({
  'Book_Name': 'string',
  'guten_genre': 'category',
  'Author_Name': 'string',
  'content': 'string'
})
corpus.dropna(inplace=True)
corpus

,Book_Name,guten_genre,Author_Name,content,tokens
book_id,,,,,
pg10067,The Mystery of the Boule Cabinet: A Detective ...,Detective and Mystery,Stevenson| Burton Egbert,"A Detective Story A.B.M. Fellow-Sherlockian ""H...","['A', 'Detective', 'Story', 'A.B.M', '.', 'Fel..."
pg1032,The Pupil,Literary,James| Henry,This edition first published 1916 The text fol...,"['This', 'edition', 'first', 'published', '191..."
pg10379,At Love's Cost,Literary,Garvice| Charles,"""Until this moment I have never fully realised...","['``', 'Until', 'this', 'moment', 'I', 'have',..."
pg10473,The Heart of the Range,Western Stories,White| William Patterson,"""The Rider of Golden Bar,"" ""Hidden Trails,"" ""L...","['``', 'The', 'Rider', 'of', 'Golden', 'Bar', ..."
pg10812,The Worshipper of the Image,Literary,Gallienne| Richard Le,The Worshipper of the Image Evening was in the...,"['The', 'Worshipper', 'of', 'the', 'Image', 'E..."
...,...,...,...,...,...
pg766DickensDavidCopfld,David Copperfield,Literary,Dickens| Charles,I do not find it easy to get sufficiently far ...,"['I', 'do', 'not', 'find', 'it', 'easy', 'to',..."
pg786DickensHardTimes,Hard Times,Literary,Dickens| Charles,The One Thing Needful Murdering the Innocents ...,"['The', 'One', 'Thing', 'Needful', 'Murdering'..."
pg834DoyleMemoirsSherlk,Memoirs of Shelock Holmes,Detective and Mystery,Connan| Doyle,"""I am afraid, Watson, that I shall have to go,...","['``', 'I', 'am', 'afraid', ',', 'Watson', ','..."


In [4]:
len(corpus.index)

994

In [5]:
corpus.astype

<bound method NDFrame.astype of                                                                  Book_Name  \
book_id                                                                      
pg10067                  The Mystery of the Boule Cabinet: A Detective ...   
pg1032                                                           The Pupil   
pg10379                                                     At Love's Cost   
pg10473                                             The Heart of the Range   
pg10812                                        The Worshipper of the Image   
...                                                                    ...   
pg766DickensDavidCopfld                                  David Copperfield   
pg786DickensHardTimes                                           Hard Times   
pg834DoyleMemoirsSherlk                          Memoirs of Shelock Holmes   
pg863Agatha1                               The Mysterious Affair at Styles   
pg98DickensTaleCities           

In [42]:
for index, book in corpus.iterrows():
    rng = round(len(str(corpus.loc[index,"content"]).split())*(4/5))
    cli = round(len(str(corpus.loc[index,"content"]).split())*(1/10)) 
    a = corpus.loc[index,"content"].split()[rng:rng+cli]
    str1 = " "
    corpus.loc[index, "climax"] = str1.join(a)
    str1 = " "

In [50]:
print(corpus.head(1)["climax"].to_numpy()[0].split())

['little', 'account.', 'I', 'have', 'a', 'profound', 'admiration', 'for', 'your', 'police;', 'the', 'results', 'which', 'they', 'accomplish', 'are', 'wonderful,', 'when', 'one', 'considers', 'the', 'difficulties', 'under', 'which', 'they', 'labour."', 'He', 'spoke', 'with', 'an', 'accent', 'so', 'sincere', 'that', 'I', 'was', 'almost', 'convinced', 'he', 'meant', 'every', 'word', 'of', 'it;', 'but', 'Godfrey', 'only', 'smiled.', '"It', 'is', 'a', 'proverb,"', 'he', 'said,', '"that', 'the', 'French', 'police', 'are', 'the', 'best', 'in', 'the', 'world.', 'You,', 'no', 'doubt,', 'have', 'a', 'theory', 'in', 'regard', 'to', 'the', 'death', 'of', 'these', 'men?"', '"I', 'fear', 'it', 'is', 'impossible,', 'sir,"', 'said', 'M.', 'Pigot,', 'regretfully,', '"to', 'answer', 'that', 'question', 'at', 'present,', 'or', 'to', 'discuss', 'this', 'case', 'with', 'you.', 'I', 'have', 'my', 'report', 'first', 'to', 'make', 'to', 'the', 'chief', 'of', 'your', 'detective', 'bureau.', 'To-morrow', 'I', '

In [57]:
X = corpus['climax']

y = corpus['guten_genre']

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0, stratify=y)

#y_train = encoder.fit_transform(y_train)
#y_test = encoder.fit_transform(y_test)

# Count Vectors
### Source: https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

In [59]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(corpus['climax'])

xtrain_count =  count_vect.transform(X_train)
xtest_count =  count_vect.transform(X_test)

# TF-IDF

In [60]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=500)
tfidf_vect.fit(corpus['climax'])
xtrain_tfidf =  tfidf_vect.transform(X_train)
xvalid_tfidf =  tfidf_vect.transform(X_test)

tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=500)
tfidf_vect_ngram.fit(corpus['climax'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(X_train)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=500)
tfidf_vect_ngram_chars.fit(corpus['climax'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_train) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(X_test) 

C:\Users\Rob\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


# Latent Dirichlet Allocation

In [61]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [78]:
list1 = corpus['climax'].tolist()

In [80]:
model = gensim.models.Word2Vec(list1, min_count=1,size=300,workers=4)

In [84]:
model.wv.rank

<bound method BaseKeyedVectors.rank of <gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x00000223F6C991C8>>


In [68]:
col_one_list = corpus['climax'].tolist()